In [ ]:
from sklearn.preprocessing import LabelEncoder
import sklearn
import tensorflow as tf
from tensorflow import keras as k
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import json
import ast
import joblib

Importing and organizing the raw data 

In [8]:
# import os
# import json
# data = list()
# for file in os.listdir("./data"):
##     if file != "results_20000-24861.json":
#     with open("./data/"+file, 'rb') as f:
#         print(f"Read {file}")
#         data_ = json.load(f)
#         data += data_
##     else:
##         continue

In [9]:
# records = [{"User": record.get('user_id'), 
#            "ISBN": record.get('book_title'),
#            "RatingOf5": record.get('stars'),
#            "Genres": record.get('genres')
#           } for record in data]
# df = pd.DataFrame(records)
# df.to_csv("USER_ISBN_RATING_GENRES.csv")

In [8]:
df = pd.read_csv("../USER_ISBN_RATING_GENRES.csv")
df.drop(columns=["Unnamed: 0"], inplace = True)

In [10]:
print("Total numer of unique books: ", len(df["ISBN"].unique()))
print("Total numer of unique users: ", len(df["User"].unique()))
print("Total numer of reviews: ", len(df))
print("Total numer of positive reviews: ", len(df[df["RatingOf5"] > 3]))
print("Total numer of non-positive reviews: ", len(df[df["RatingOf5"] < 3]))
print("Total numer of moderate reviews: ", len(df[df["RatingOf5"] == 3]))
print("Total numer of  non-reviews: ", sum(df["RatingOf5"].isna()))

Total numer of unique books:  57198
Total numer of unique users:  706880
Total numer of reviews:  1733561
Total numer of positive reviews:  936796
Total numer of non-positive reviews:  201628
Total numer of moderate reviews:  372374
Total numer of  non-reviews:  222763


To keep users anonymous, I'll label encode them as IDs

In [11]:
le = LabelEncoder()
le.fit(df["User"])
transformed = le.transform(df["User"])
len(transformed), len(np.unique(transformed, return_counts=False))

(1733561, 706880)

In [12]:
joblib.dump(le, "fitted_label_encoder.pkl")

['fitted_label_encoder.pkl']

In [13]:
le = joblib.load("fitted_label_encoder.pkl")
transformed = le.transform(df["User"])

In [14]:
df["User"] = transformed
df["User"].head()

0    423114
1    241063
2    659623
3    557019
4     44930
Name: User, dtype: int32

In [59]:
df2 = df.copy()

**I'm going to reserve a random sample of 10 observations to test recommendations as observations that the model has not seen before when developing the production package.**

In [16]:
sample = df.groupby("ISBN").count()[df.groupby("ISBN").count()["User"] > 5]["User"].sort_values(ascending=False).sample(20).index
loop = True
while True:
    new_sample=df[df["ISBN"].isin(sample)].sample(10)['ISBN']
    if len(new_sample.unique()) == 10:
        sample_to_keep = new_sample
        break

In [17]:
sample = df[df.index.isin(sample_to_keep.index)][['User', 'ISBN']]
index_to_drop = sample.index
sample.to_csv('test.csv', index=False)
df.drop(index=index_to_drop, axis=0, inplace=True)

In [18]:
df[df.index.isin(index_to_drop)]

,User,ISBN,RatingOf5,Genres


In [19]:
pd.read_csv('test.csv')

,User,ISBN
0,36183,Voleurs de plage
1,542310,Deadly Grounds Sydney Bryant 2
2,651722,Jumpin' Joe: The Jozef Sabovcik Story
3,400575,The Church Cat Abroad
4,267937,El hombre en el castillo
5,496853,Core MySQL: The Serious Developer's Guide
6,278028,Lord of Fire Lords and Ladies 1
7,202310,The Beans of Egypt Maine
8,627962,Poet and Dancer
9,589726,Thinner


In [64]:
df[["User", "ISBN"]].to_csv('train_data.csv', index=False)

In [65]:
pd.read_csv('train_data.csv', nrows=5)

,User,ISBN
0,423114,Kiss Hollywood Goodbye
1,241063,Kiss Hollywood Goodbye
2,659623,Kiss Hollywood Goodbye
3,557019,Kiss Hollywood Goodbye
4,44930,Kiss Hollywood Goodbye


#### Reducing dimentionality

With ~ 700k users and 60k books, the embeddings for the two embedding "towers" that the recommender model is going to be based on will generately an extremely large model. Practically, I would like to demonstrate the end-to-end data science pipeline by being able to package my model and make it available publicly for my API to access as well as for those interested which is why reduce the recommendations to those users who given more than 2 review.

In [20]:
grouped_data = df.groupby("User").count()
grouped_data[grouped_data["ISBN"] > 2]['ISBN']

User
0         42
7          3
8          4
25         3
31         3
          ..
706863    12
706867     6
706870     6
706871     4
706879     4
Name: ISBN, Length: 130887, dtype: int64

In [21]:
len(df)

1733551

In [22]:
idx_to_keep = grouped_data[grouped_data["ISBN"] > 2].index
set(idx_to_keep) == set(df[df["User"].isin(idx_to_keep)]["User"])

True

At this point we have an index of user that have more than 2 reviews (of different books). Our new DataFrame object will only comprise those users and their respectively reviewed books

In [23]:
df = df[df.User.isin(idx_to_keep)]
df.sample(5)[['User', 'ISBN']]

,User,ISBN
1245158,264027,High Fidelity
464505,479022,The Autobiography of Benjamin Franklin
76498,165464,The Fencing Master
1013314,139514,'Salem's Lot
268727,550069,Salem Witch Trials: Alliance Project 1


In [24]:
# also ensure that the grouping and querying was sane
print("# of unique users remaining: ", len(df.groupby('User').count()))

# also check out how many unique books were lost in the process of trimming 
# down the dataset (only about 500)

print("# of unique books remaining: ", len(df.groupby('ISBN').count()))

    

# of unique users remaining:  130887
# of unique books remaining:  56669


In [25]:
# we can label encode the book ID's if we so wish
le_isbn = LabelEncoder()
df["ISBN"] = le_isbn.fit_transform(df["ISBN"])

C:\Users\Moham\AppData\Local\Temp\ipykernel_22868\42863754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ISBN"] = le_isbn.fit_transform(df["ISBN"])


Setting up tensors to batch for purposes of training the embeddings 

In [26]:
books = tf.data.Dataset.from_tensor_slices(
    df["ISBN"].astype('str').values).shuffle(128) # TODO: config (shuffle seed + size)

ratings = tf.data.Dataset.from_tensor_slices(
    df[["ISBN", "User"]].astype('str').values).shuffle(128) # TODO: config (shuffle seed + size)

In [27]:
for batch in ratings.batch(batch_size=10).take(1):
    break

# [[Book title, User ID],]
[[batch[i].numpy()[j].decode() for j in range(2)] for i in range(10)]

[['22378', '423114'],
 ['50409', '264027'],
 ['51897', '216574'],
 ['50409', '377761'],
 ['50409', '111376'],
 ['32867', '111877'],
 ['22378', '36177'],
 ['14058', '670849'],
 ['22378', '349993'],
 ['25405', '285275']]

String look up creates a vocabulary dictionary of string-value pairs since I'll be embedding my ISBN's and users ID's

In [38]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x[1]))

book_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
book_titles_vocabulary.adapt(books)

In [40]:
np.save("user_ids_vocabulary", user_ids_vocabulary.get_weights())
np.save("book_titles_vocabulary", book_titles_vocabulary.get_weights())

In [41]:
new_book_title_stringlookup = tf.keras.layers.StringLookup(mask_token=None)
new_book_title_stringlookup.set_weights(np.load("book_titles_vocabulary.npy", allow_pickle=True))
new_user_id_stringlookup = tf.keras.layers.StringLookup(mask_token=None)
new_user_id_stringlookup.set_weights(np.load("user_ids_vocabulary.npy", allow_pickle=True))

In [42]:
# compare vocab against original data as a sanity check
print("Unique User ID counts: ", len(df["User"].unique()), new_user_id_stringlookup.vocabulary_size())
print("Unique ISBN counts: ", len(df["ISBN"].unique()), new_book_title_stringlookup.vocabulary_size())

Unique User ID counts:  130887 130888
Unique ISBN counts:  56669 56670


The difference in "vocab" counts results from the [UNK] (i.e. unknown) token. 

Testing stringlookup on the saved sample:

In [43]:
len(new_book_title_stringlookup.get_vocabulary())

56670

In [44]:
new_book_title_stringlookup.call(tf.constant(sample['ISBN'].astype(str))), sample['ISBN']

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>,
 94045                               Voleurs de plage
 587383                Deadly Grounds Sydney Bryant 2
 812079         Jumpin' Joe: The Jozef Sabovcik Story
 1088663                        The Church Cat Abroad
 1309076                     El hombre en el castillo
 1413731    Core MySQL: The Serious Developer's Guide
 1636847              Lord of Fire Lords and Ladies 1
 1691249                     The Beans of Egypt Maine
 1716389                              Poet and Dancer
 1728612                                      Thinner
 Name: ISBN, dtype: object)

In [45]:
new_user_id_stringlookup.call(tf.constant(sample['User'].astype(str))), sample['User']

(<tf.Tensor: shape=(10,), dtype=int64, numpy=
 array([110217,  22004,      0,      0,    256,      0,  11467,      0,
         30130,    347], dtype=int64)>,
 94045       36183
 587383     542310
 812079     651722
 1088663    400575
 1309076    267937
 1413731    496853
 1636847    278028
 1691249    202310
 1716389    627962
 1728612    589726
 Name: User, dtype: int32)

Several user_ids have a string lookup of 0 which stands for unknown meaning they were likely dropped during the attemp at dimentionality reduction (i.e. these users had only ever reviewed one/two book)

In [46]:
idx = [0, 3, 8]

# this is a bit of  confusing one-liner that simply attempts to search for the indices of users that 
# had the string lookup assign them a value of 0 and thus are the suspicions confirmed that
# they've been dropped during the dimentionality reduction step since they're users with less than 
# 2 total reviews.

assert len(df[df.index.isin(sample[sample['User'].reset_index().index.isin(idx)].index)]['User'].unique()) > 0

AssertionError: 



I'll be doing tests for label encoders and string lookups on the books columns and not the users columns before deployment 
since their info is confidential, otherwise, for testing purposes, the sample taken isn't valid and would fail the tests

#### Modelling

The two tower model has user features and movie features as embeddings and uses implicit assumptions as in books read being a positive example and those not read being an negative one. TFRS Retrieval handles this through a top K factorization which compares accuracy `tf.keras.metrics.TopKCategoricalAccuracy(k=K)` of books predicted to be by the queries user vs actual books read.



In [51]:
BATCH_SIZE = 1024*8
EPOCHS = 10

In [95]:
class TwoTowerModel(tfrs.Model):
    def __init__(
      self,
      user_model: tf.keras.Model,
      book_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
        super(TwoTowerModel, self).__init__()
    
        # embedding representations
        self.user_model = user_model
        self.book_model = book_model
        # this will receive the computed losses by using the retrieval class to calculate the FactorizedTopK
        self.task = task 
    
    def compute_loss(self, features, training=False) -> tf.Tensor:
        # Define how the loss is computed.
        user_embeddings = self.user_model(features[:, 1])
        book_embeddings = self.book_model(features[:, 0])
        
#         print(user_embeddings.shape)
        
        return self.task(user_embeddings, book_embeddings, compute_metrics=False)

In [96]:
user_model = tf.keras.Sequential([
    new_user_id_stringlookup,
    tf.keras.layers.Embedding(new_user_id_stringlookup.vocabulary_size()+1 , 32)
])

book_model = tf.keras.Sequential([
    new_book_title_stringlookup,
    tf.keras.layers.Embedding(new_book_title_stringlookup.vocabulary_size()+1, 32)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    books.batch(256).map(book_model)
  )
)

In [97]:
tt_model = TwoTowerModel(user_model, book_model, task)
tt_model.compile(optimizer=tf.keras.optimizers.Adam(0.1))

In [98]:
for i in ratings.take(1):
    print(i)
    break

tf.Tensor([b'14058' b'670849'], shape=(2,), dtype=string)


In [99]:
tt_model.fit(ratings.batch(BATCH_SIZE), epochs=10)

Epoch 1/10
129/129 [==============================] - 4s 29ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 75616.6461 - regularization_loss: 0.0000e+00 - total_loss: 75616.6461
Epoch 2/10
129/129 [==============================] - 4s 29ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 101869.5806 - regularization_loss: 0.0000e+00 - total_loss: 101869.5806
Epoch 3/10
129/129 [==============================] - 4s 29ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000

The brute force approach goes through every index to get the top K recommendation. This is considered tedious and the alternative scalable nearest neighbours approach provides a viable alternative since the latter utilizes compressed vectors. 

In [100]:
indexer = tfrs.layers.factorized_top_k.BruteForce(tt_model.user_model, k = 100)
indexer.index_from_dataset(
    books.batch(64).map(lambda title: (title, tt_model.book_model(title))))

Let's get some recommendations.

In [101]:
print(idx) # users that the string lookupand hence model doesn't know and should give then random recommendations
print(sample['User'].values)

[0, 3, 8]
[ 36183 542310 651722 400575 267937 496853 278028 202310 627962 589726]


In [107]:
titles = indexer(tf.constant(["703730"]))
_, titles = indexer(tf.constant(["703730"]))
# the 5 unique recommendations indicates the retriever model along isn't enough
np.unique([titles.numpy()[0][i].decode() for i in range(len(titles.numpy()[0]))]) 

array(['15006', '40107', '40442', '52155', '53676'], dtype='<U5')

In [109]:
%%timeit _, titles = indexer(tf.constant(["3"]))
_, titles = indexer(tf.constant(["3"]))
set([titles.numpy()[0][i].decode() for i in range(len(titles.numpy()[0]))])

10 ms ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The time taken is quite slow for a live model which is why a ScaNN alogorithm can be used which approximates recommendations user a nearest-neighbours approach.

As a sanity check, I'll get a couple of unknow users' recommendations

In [110]:
assert ("683095" or "47561") not in new_user_id_stringlookup.get_vocabulary()

In [111]:
for user in ['683095', '47561 ']:
    _, titles = indexer(tf.constant([user]))
    print(user, ": ", set([titles.numpy()[0][i].decode() for i in range(len(titles.numpy()[0]))]))

683095 :  {'32926', '21845', '11710', '42079', '30917', '4823', '20803', '54757'}
47561  :  {'32926', '21845', '11710', '42079', '30917', '4823', '20803', '54757'}


The similarity in recommendations to unknown indicates that all unknown users are given the same random recommendation.

**Making sense of the recommendations requires analyzing the genres of book user "100023" read and liked.**

In [113]:
read_genres = set()
user_read_genres = df2[df2["User"]==55862]["Genres"].unique()
user_read_genres = [ast.literal_eval(user_read_genres[i]) for i in range(len(user_read_genres))];
[[read_genres.add(i) for i in element] for element in user_read_genres];
list(read_genres)

['Gothic',
 'Dogs',
 'Adventure',
 'Suspense',
 'Young Adult',
 'Womens',
 'Banned Books',
 'How To',
 'Nonfiction',
 'Crime',
 'Mystery Thriller',
 'Family',
 'Memoir',
 'Short Story Collection',
 'Womens Fiction',
 'Ireland',
 'Poetry',
 'Philosophy',
 'Adult',
 'Health',
 'Outdoors',
 'The United States Of America',
 'European Literature',
 '20th Century',
 'Supernatural',
 'Detective',
 'France',
 'Personal Development',
 'Nature',
 'Witches',
 'Fiction',
 'Kids',
 'Scotland',
 'Book Club',
 'Cultural',
 'Realistic Fiction',
 'Young Readers',
 'Canada',
 'Paranormal',
 'Modern Classics',
 'Animals',
 'Academic',
 'Travel',
 'Coming Of Age',
 'Female Authors',
 'Chapter Books',
 'School',
 'Gothic Romance',
 'Short Stories',
 'Psychological Thriller',
 'Juvenile',
 'Cozy Mystery',
 'Emergency Services',
 'Audiobook',
 'Spirituality',
 'Self Help',
 'Childrens',
 'Historical',
 'Autobiography',
 'Diets',
 'Psychology',
 'Mystery',
 'Biography Memoir',
 'Romantic Suspense',
 'Biograph

In [114]:
user_recommended_books = ['Brennen muss Salem',
 'How to Know God: The Yoga Aphorisms of Patanjali',
 "It's Not Mean If It's True: More Trials from My Queer Life",
 'Night Judgement At Sinos A Novel',
 'The House on the Point: A Tribute to Franklin W Dixon and The Hardy Boys',
 'Tularosa Kevin Kerney 1']

In [115]:
recommendations = df2[df2["ISBN"].isin(user_recommended_books)]['Genres'].unique()
recommendations = [ast.literal_eval(recommendations[i]) for i in range(len(recommendations))]

In [116]:
rec_genres = set()
[[rec_genres.add(i) for i in element] for element in recommendations];
rec_genres

{'Action',
 'Adult',
 'Adventure',
 'Audiobook',
 'Autobiography',
 'Buddhism',
 'Classics',
 'Crime',
 'Cultural',
 'Eastern Philosophy',
 'Emergency Services',
 'Espionage',
 'Essays',
 'Fantasy',
 'Fiction',
 'Gay',
 'Hinduism',
 'Horror',
 'Humor',
 'India',
 'LGBT',
 'Memoir',
 'Mystery',
 'Mystery Thriller',
 'Nonfiction',
 'Novels',
 'Paranormal',
 'Philosophy',
 'Police',
 'Psychology',
 'Queer',
 'Religion',
 'Spirituality',
 'Spy Thriller',
 'Supernatural',
 'Suspense',
 'Thriller',
 'Vampires',
 'Westerns',
 'Writing'}

In [117]:
read_genres.intersection(rec_genres)

{'Adult',
 'Adventure',
 'Audiobook',
 'Autobiography',
 'Buddhism',
 'Classics',
 'Crime',
 'Cultural',
 'Emergency Services',
 'Fantasy',
 'Fiction',
 'Humor',
 'Memoir',
 'Mystery',
 'Mystery Thriller',
 'Nonfiction',
 'Novels',
 'Paranormal',
 'Philosophy',
 'Police',
 'Psychology',
 'Religion',
 'Spirituality',
 'Supernatural',
 'Suspense',
 'Thriller'}

The model was able to pickup on a lot of user 55862's preferences such as, however it does miss others. A more detailed look at book ratings for the recommended and missed genres is required. Further, users with a lesser number of read books require analyzing.

##### Packing the model

In [118]:
model_name = "model"
version =  "_v0.0.1"
zipped_model_name = "zipped_model"

In [119]:
import os
import shutil
# The use of the nested dirctory below is intended to ease zipping and unizpping later on
tf.saved_model.save(indexer, os.path.join(f"./{model_name}{version}", f"{model_name}{version}"))

path = os.path.join(
            f"{model_name}{version}",
            f"{model_name}{version}",
            "assets",
            "placeholder")
open(path, 'a').close()

INFO:tensorflow:Assets written to: ./model_v0.0.1\model_v0.0.1\assets


INFO:tensorflow:Assets written to: ./model_v0.0.1\model_v0.0.1\assets


In [120]:
loaded = tf.saved_model.load(os.path.join(f"./{model_name}{version}", f"{model_name}{version}"))

In [121]:
type(indexer), type(loaded)

(tensorflow_recommenders.layers.factorized_top_k.BruteForce,
 tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject)

In [122]:
_, titles = loaded(tf.constant(["55862"]))
set([titles.numpy()[0][i].decode() for i in range(len(titles.numpy()[0]))])

{'10753', '12683', '12921', '42062', '47722', '5963'}

In [123]:
_, titles = indexer(tf.constant(["703730"]))
set([titles.numpy()[0][i].decode() for i in range(len(titles.numpy()[0]))])

{'18684', '28325', '32678', '44154', '5252'}

Given the large model size, it's relevant to compress it for purposes of packaging since pruning using keras pruning tools wouldn't work on this custom tfrs Model. The target's to get it under 50 MiB to maintain on GitHub. Unzipping and loading the model takes a long time (500 ms) and does not make it suitable for live prediction meaning the model should be unzipped ideally as soon as the application goes live. Contanier orchastraion can facilitate the takedown and setup of pods where time delay in unzipping is no longer an encumbrance>

In [172]:
import os
import zipfile
import shutil
def zip_directory(folder_path: str, zip_path:str, zip: bool = True, test: bool = False):
    if zip:
        with zipfile.ZipFile(zip_path, mode='w', compression=zipfile.ZIP_DEFLATED) as zipf:
            len_dir_path = len(folder_path)
            for root, _, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, file_path[len_dir_path:])
            zipf.close()
            shutil.rmtree(folder_path)
            if test:
                result = (os.path.isdir(folder_path) and 
                          os.path.isfile(zip_path)) # false and true
                return result == False and True
            
    else:
        if test:
            extract_path = os.mkdir(folder_path)
        else:
            extract_path = None
        with zipfile.ZipFile(file=zip_path, mode="r") as f:
            f.extractall()
            f.close()
            os.remove(zip_path)
            if test:
                result = (os.path.isfile(zip_path) and 
                            os.path.isdir(folder_path)) # false and true
                return result == False and True

Test zipping

In [173]:
zip_directory(folder_path=f"./{model_name}{version}", 
              zip_path=f"{zipped_model_name}{version}.zip",
              zip=True,
              test = True)

True

Test unzipping

In [174]:
zip_directory(folder_path=f"./{model_name}{version}", 
              zip_path=f"{zipped_model_name}{version}.zip",
              zip=False,
              test = True)

True

Test the unzipped model

In [126]:
loaded = tf.saved_model.load(f"{model_name}{version}")

In [128]:
_, titles = loaded(tf.constant(["1"]))
set([titles.numpy()[0][i].decode() for i in range(len(titles.numpy()[0]))])

{'52727', '53615'}